In [1]:
%run ../Regression_Analysis/SLP_REGRESSION.ipynb

Epoch 1 : loss = 35.182, accuracy = 0.547/0.803
Epoch 2 : loss = 8.874, accuracy = 0.814/0.809
Epoch 3 : loss = 8.134, accuracy = 0.805/0.803
Epoch 4 : loss = 8.017, accuracy = 0.802/0.804
Epoch 5 : loss = 7.924, accuracy = 0.803/0.802
Epoch 6 : loss = 7.843, accuracy = 0.802/0.803
Epoch 7 : loss = 7.767, accuracy = 0.802/0.804
Epoch 8 : loss = 7.697, accuracy = 0.802/0.806
Epoch 9 : loss = 7.634, accuracy = 0.805/0.804
Epoch 10 : loss = 7.574, accuracy = 0.804/0.805

 Final Test : final accuracy = 0.805
[[0.55842942]
 [0.76572744]
 [0.00429812]
 [2.24857982]
 [1.78899292]
 [0.66286655]
 [2.723841  ]
 [0.65809305]
 [0.54706306]
 [1.1225188 ]]
[4.52425871]
Epoch 1 : loss = 7.521, accuracy = 0.803/0.757
Epoch 2 : loss = 6.212, accuracy = 0.817/0.803
Epoch 3 : loss = 5.859, accuracy = 0.824/0.835
Epoch 4 : loss = 5.598, accuracy = 0.828/0.838
Epoch 5 : loss = 5.440, accuracy = 0.830/0.838
Epoch 6 : loss = 5.240, accuracy = 0.834/0.796
Epoch 7 : loss = 5.201, accuracy = 0.834/0.850
Epoch 8

## SLP REGRESSION function call relation

<img src = "KakaoTalk_20220501_175228225.jpg" width = "700px">

In [2]:
def slp_binary_classification(epoch_count = 10, mb_size = 10, report = 1, adjust_ratio = False):
    load_pulsar_dataset(adjust_ratio)
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [3]:
def load_pulsar_dataset(adjust_ratio):
    pulsars, stars = [], []
    
    with open('./pulsar_data/Pulsar.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        
        for row in csvreader:
            if row[8] == '1':
                pulsars.append(row)
            else:
                stars.append(row)
            
        
        global data, input_cnt, output_cnt
        input_cnt = 8
        output_cnt = 1
        
        star_cnt, pulsar_cnt = len(stars), len(pulsars)
        
        if adjust_ratio:
            data = np.zeros([2*star_cnt, 9])
            data[0:star_cnt, :] = np.asarray(stars, dtype = float)
            
            for n in range(star_cnt):
                data[star_cnt + n] = np.asarray(pulsars[n % pulsar_cnt], dtype = float)
            
        else:
            data = np.zeros([star_cnt + pulsar_cnt, 9])
            data[0:star_cnt, :] = np.asarray(stars, dtype = float)
            data[star_cnt:, :] = np.asarray(pulsars, dtype = float)

In [4]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses = []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, _  = run_train(train_x, train_y)
            losses.append(loss)
            
        if (report > 0) and ((epoch+1) % report == 0):
            acc = run_test(test_x, test_y)
            acc_str = ','.join(['%5.3f']*4) % tuple(acc)
            print('Epoch {} : loss = {:5.3f}, result = {}' .format(epoch + 1, np.mean(losses), acc_str))
            
    acc = run_test(test_x, test_y)
    acc_str = ','.join(['%5.3f']*4) % tuple(acc)
    print('\n Final Test : final result = {}' .format(acc_str))

In [5]:
def forward_postproc(output, y):
    entropy = sigmoid_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy)
    
    return loss, [y, output, entropy]


def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0/np.prod(entropy.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

In [6]:
def eval_accuracy(output, y):
    est_yes = np.greater(output, 0)
    ans_yes = np.greater(y, 0.5)
    est_no = np.logical_not(est_yes)
    ans_no = np.logical_not(ans_yes)
    
    tp = np.sum(np.logical_and(est_yes, ans_yes))
    fp = np.sum(np.logical_and(est_yes, ans_no))
    fn = np.sum(np.logical_and(est_no, ans_yes))
    tn = np.sum(np.logical_and(est_no, ans_no))
    
    accuracy = safe_div(tp + tn, tp + tn + fp + fn)
    precision = safe_div(tp, tp + fp)
    recall = safe_div(tp, tp + fn)
    f1 = 2 * safe_div(recall * precision, recall + precision)
    
    return [accuracy, precision, recall, f1]

def safe_div(p, q):
    p, q = float(p), float(q)
    
    if np.abs(q) < 1.0e-20:
        return np.sign(p)
    
    return p / q

In [7]:
def relu(x):
    return np.maximum(x, 0)

def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))

def sigmoid_derv(x, y):
    return y*(1-y)

def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)

In [8]:
slp_binary_classification()

Epoch 1 : loss = 15.074, result = 0.963,0.861,0.725,0.787
Epoch 2 : loss = 12.927, result = 0.948,0.674,0.862,0.757
Epoch 3 : loss = 11.163, result = 0.971,0.987,0.701,0.820
Epoch 4 : loss = 12.410, result = 0.956,0.989,0.530,0.690
Epoch 5 : loss = 11.734, result = 0.964,0.986,0.626,0.766
Epoch 6 : loss = 10.484, result = 0.969,0.966,0.686,0.802
Epoch 7 : loss = 12.039, result = 0.976,0.981,0.754,0.853
Epoch 8 : loss = 12.412, result = 0.973,0.919,0.781,0.845
Epoch 9 : loss = 10.885, result = 0.963,0.986,0.611,0.754
Epoch 10 : loss = 12.293, result = 0.975,0.935,0.781,0.852

 Final Test : final result = 0.975,0.935,0.781,0.852


In [9]:
slp_binary_classification(adjust_ratio = True)

Epoch 1 : loss = 39.136, result = 0.909,0.909,0.910,0.909
Epoch 2 : loss = 36.505, result = 0.839,0.785,0.934,0.853
Epoch 3 : loss = 35.209, result = 0.916,0.967,0.862,0.911
Epoch 4 : loss = 35.107, result = 0.888,0.987,0.787,0.876
Epoch 5 : loss = 35.467, result = 0.922,0.961,0.880,0.919
Epoch 6 : loss = 35.625, result = 0.877,0.993,0.760,0.861
Epoch 7 : loss = 32.221, result = 0.913,0.927,0.898,0.912
Epoch 8 : loss = 31.688, result = 0.891,0.983,0.796,0.879
Epoch 9 : loss = 33.389, result = 0.896,0.876,0.922,0.899
Epoch 10 : loss = 33.949, result = 0.914,0.916,0.912,0.914

 Final Test : final result = 0.914,0.916,0.912,0.914
